In [1]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

# query string 문자열을 dict 선언
payload = {
    'query': '파이썬',
    'display': 10,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/book.json' #?query=파이썬&display=100&sort=sim

# requests get(url, params, headers) 요청 
res = requests.get(url, params=payload, headers=headers)
# json() 함수로 응답 결과 가져오기
items_data = res.json()['items']

pprint(items_data)

[{'author': '박응용',
  'description': '프로그래밍 분야 8년 연속 베스트셀러!\n'
                 '《Do it! 점프 투 파이썬》 전면 개정 2판 출시!\n'
                 '\n'
                 '중고등학생도, 비전공자도, 직장인도 프로그래밍에 눈뜨게 만든 바로 그 책이 전면 개정 2판으로 새롭게 '
                 '태어났다! 챗GPT를 시작으로 펼쳐진 생성 AI 시대에 맞춰 설명과 예제를 다듬고, 최신 경향과 심화 내용을 '
                 '보충했다. 또한 이번 개정 2판도 50만 코딩 유튜버인 조코딩과 협업을 통해 유튜브 동영상을 제공해 파이썬을 '
                 '더 쉽게 공부할 수 있다.\n'
                 '\n'
                 '8년 연속 베스트셀러! 위키독스 누적 방문 300만! 독자의 입에서 입으로 전해진 추천과 수많은 대학 및 '
                 '학원의 교재 채택을 통해 검증은 이미 끝났다. 코딩을 처음 배우는 중고등학생부터 코딩 소양을 기르려는 '
                 '비전공자, 자기계발에 진심인 직장인까지! 이 책과 함께 파이썬 프로그래밍의 세계로 ‘점프’해 보자!',
  'discount': '19800',
  'image': 'https://shopping-phinf.pstatic.net/main_4035408/40354085633.20230927071024.jpg',
  'isbn': '9791163034735',
  'link': 'https://search.shopping.naver.com/book/catalog/40354085633',
  'pubdate': '20230615',
  'publisher': '이지스퍼블리싱',
  'title': 'Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서)'},
 {'author': '

/Users/joy/dev/python_Rookies/Python_WebScraping_Analysis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
